In [1]:
import plotly
import plotly.graph_objs as go
import plotly.express as px
import numpy as np
import pandas as pd
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)
plotly.io.templates.default = "none"

### Данные

Воспользуемся данными с сайта [Федеральной службы государственной статистики](http://www.gks.ru/wps/wcm/connect/rosstat_main/rosstat/ru/). Нам необходима информация по регионам России за несколько лет. Составим обучающую выборку из таблиц:

* [Данные о доходах населения](http://bi.gks.ru:8080/DDB/showcharts.jsp?report=UR001_1p&project=BIPortal_cen_3.bip) за 1995-2009 года
* [Данные об уровне преступности](http://bi.gks.ru:8080/DDB/showcharts.jsp?report=prest01&lang=ru&project=BIPortal_cen_2.bip) за 1996-2010 года
* [Данные о безработице](http://bi.gks.ru:8080/DDB/showcharts.jsp?report=TR000_2p&lang=ru&project=BIPortal_cen_2.bip) за 1996-2010 года

Функционал сайт позволяет скачать таблицы в формате `xls`. К сожалению, при импорте таблиц возникает проблема с кодировками, поэтому откроем таблицы в excel и сохраним их в формате `.xlsx`, убрав лишние столбцы/строки содержашие названия переменных.   

In [2]:
# Загружаем таблицу с уровнем преступности
crime = pd.read_excel('./data/crime.xlsx', index_col=None, header=None)
# уберем ненужные поля и выставим названия переменных
columns = [12*i for i in range(15)]
crime = crime[columns].iloc[2:]
crime.columns = ['Регион'] + [1996 + i for i in range(14)]
crime.set_index('Регион', inplace = True)
# Развернем таблицу
crime = pd.DataFrame(crime.stack()).reset_index()
crime.columns = ['Регион', 'Год', 'Число преступлений']
crime['Число преступлений'] = crime['Число преступлений'].astype(float)
crime.head()

,Регион,Год,Число преступлений
0,Белгородская область,1996,16759.0
1,Белгородская область,1997,15467.0
2,Белгородская область,1998,17172.0
3,Белгородская область,1999,20866.0
4,Белгородская область,2000,19907.0


In [3]:
income = pd.read_excel('./data/income.xlsx')
# Развернем таблицу
income = pd.DataFrame(income.stack()).reset_index()
income.columns = ['Регион', 'Год', 'Средний доход']
# Уберем года до 1998, топому что в датасете они указаны в тысячах рублей
income = income[(income['Средний доход'] != ' ') & (income['Год'] >= 1999)]
income['Средний доход'] = income['Средний доход'].astype(float)
income.head()

,Регион,Год,Средний доход
4,Белгородская область,1999,1187.1
5,Белгородская область,2000,1554.5
6,Белгородская область,2001,2121.4
7,Белгородская область,2002,2762.3
8,Белгородская область,2003,3357.4


In [4]:
unemployment = pd.read_excel('./data/unemployment.xlsx').iloc[1:]
temp = pd.DataFrame(index=unemployment.index, columns=[1996 + i for i in range(15)])
for year in range(15):
    temp[1996 + year] = unemployment.iloc[:, 12*year:12*(year+1)].sum(axis = 1)*1000
unemployment = pd.DataFrame(temp.stack()).reset_index()
unemployment.columns = ['Регион', 'Год', 'Число безработных']
unemployment.head()

,Регион,Год,Число безработных
0,Белгородская область,1996,97200.0
1,Белгородская область,1997,112600.0
2,Белгородская область,1998,103900.0
3,Белгородская область,1999,132900.0
4,Белгородская область,2000,56400.0


#### Посмотрим, как меняются показатели с годами

In [5]:
region_sample = np.random.choice(crime['Регион'].unique(), size = 8, replace = False)

In [6]:
sample = crime[crime['Регион'].isin(region_sample)].copy()
px.line(sample, x = 'Год', y = 'Число преступлений', color = 'Регион')

In [7]:
sample = income[income['Регион'].isin(region_sample)].copy()
px.line(sample, x = 'Год', y = 'Средний доход', color = 'Регион')

In [8]:
sample = unemployment[unemployment['Регион'].isin(region_sample)].copy()
px.line(sample, x = 'Год', y = 'Число безработных', color = 'Регион')

#### Соеденим таблицы вместе

In [9]:
data = unemployment.merge(income, left_on = ['Регион', 'Год'], right_on = ['Регион', 'Год'])
data = data.merge(crime, left_on = ['Регион', 'Год'], right_on = ['Регион', 'Год'])
data.head()

,Регион,Год,Число безработных,Средний доход,Число преступлений
0,Белгородская область,1999,132900.0,1187.1,20866.0
1,Белгородская область,2000,56400.0,1554.5,19907.0
2,Белгородская область,2001,68200.0,2121.4,20351.0
3,Белгородская область,2002,86800.0,2762.3,18092.0
4,Белгородская область,2003,104200.0,3357.4,19123.0


In [10]:
px.scatter_matrix(
    data_frame = data,
    dimensions = ['Число безработных', 'Средний доход' , 'Число преступлений'],
    color = 'Год'
)

Из графика выше видно, что число преступлений коррелирует с числом безработных и слабо видна зависимоть от среднего дохода.

#### Построение модели.

Перейдем к построению модели. Отложим последние два года в тестовую выборку. На оставшихся данных настроим модель линейной регрессии с $L_1$ регуляризатором и случайный лес. 

Линейная модель с $L_1$ регуляризацией позволит нам определить важность переменных (переменные не влияющие на целевую имеют вес близкий к нулю).

Сравним результат прогноза моделей с простым прогнозом уровня преступности. __Baseline__ прогноз: уровень преступности в следующем году равен уровню преступности в прошлом году 

In [11]:
from sklearn.utils import shuffle

# Разделяем на train/test
data = shuffle(data)
# Нормируем показатели 
data['Число безработных'] = (data['Число безработных'] - data['Число безработных'].mean())/data['Число безработных'].std()
data['Средний доход'] = (data['Средний доход'] - data['Средний доход'].mean())/data['Средний доход'].std()
X_train = data[~data['Год'].isin([2008, 2009])][['Число безработных', 'Средний доход', 'Год', 'Регион']]
X_test = data[data['Год'].isin([2008, 2009])][['Число безработных', 'Средний доход', 'Год', 'Регион']]
y_train = data[~data['Год'].isin([2008, 2009])]['Число преступлений']
y_test = data[data['Год'].isin([2008, 2009])]['Число преступлений']

#### Зависимость числа преступлений от среднего дохода и числа безработных

In [12]:
px.scatter(data, x = 'Число безработных', y = 'Средний доход', color = 'Число преступлений')

In [13]:
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor

##### Создадим baseline прогноз

In [14]:
baseline_predict = X_train.sort_values(
    by = ['Регион', 'Год'],
    ascending=[True, False]).groupby(by = ['Регион']).head(1)
baseline_predict = baseline_predict[['Регион', 'Средний доход']]
baseline_predict.columns = ['Регион', 'Прогноз']

##### Оценим точность baseline подхода

In [15]:
from sklearn.metrics import r2_score
temp = X_test.merge(baseline_predict, left_on='Регион', right_on = 'Регион')
y = temp['Средний доход']
pred = temp['Прогноз']
print('Коэффициент детерминации R^2 на тесте = ', r2_score(y, pred))

Коэффициент детерминации R^2 на тесте =  0.7399586174516317


##### Настроим модель Lasso регрессии

Какое-то очень низкое качество на тесте

In [16]:
reg = LassoCV(cv=5, normalize=True).fit(X_train.drop(columns=['Год', 'Регион']), y_train.values)
print('Коэффициент детерминации R^2 на трейне = ', reg.score(X_train.drop(columns=['Год', 'Регион']).values, y_train))
print('Коэффициент детерминации R^2 на тесте = ', reg.score(X_test.drop(columns=['Год', 'Регион']).values, y_test))

Коэффициент детерминации R^2 на трейне =  0.38756539688386327
Коэффициент детерминации R^2 на тесте =  0.13408031357713734


Посмотрим на коэффициенты регрессии

In [17]:
reg.coef_

array([19256.82534015, 11091.68372844])

##### Настроим модель случайного леса

In [18]:
from sklearn.model_selection import GridSearchCV
rf = RandomForestRegressor(random_state=42)
param_grid = { 
    'n_estimators' : [10, 50, 100, 200, 500],
    'max_depth' : [1, 2, 3, 4, 5, 6, 7],
}
CV_rf = GridSearchCV(estimator = rf, param_grid = param_grid, cv= 5)
CV_rf.fit(X_train.drop(columns=['Год', 'Регион']), y_train)
print('Коэффициент детерминации R^2 на трейне = ', CV_rf.score(X_train.drop(columns=['Год', 'Регион']), y_train))
print('Коэффициент детерминации R^2 на тесте = ', CV_rf.score(X_test.drop(columns=['Год', 'Регион']), y_test))

Коэффициент детерминации R^2 на трейне =  0.6640710143323383
Коэффициент детерминации R^2 на тесте =  -0.009584839522634336


## Вывод

Как мы видим теория Беккера работает довольно плохо на наших данных. Самый простой прогноз: *уровень преступлности в следующем году равен уровню преступности в предыдущем году* дает значительно лучшие результаты чем модель, настроенная на обучающей выборке.

#### Улучшение прогноза модели

Если добавить в модель две независимые переменные, __Год__ и  __Регион__ качество прогноза увеличится, но все равно останется хуже baseline модели. 

In [19]:
from catboost import CatBoostRegressor
model = CatBoostRegressor(cat_features=[2,3])
model.fit(X_train, y_train, verbose = False)
preds = model.predict(X_test)
print('Коэффициент детерминации R^2 на тесте = ', r2_score(y_test, preds))

Коэффициент детерминации R^2 на тесте =  0.44632565799704826
